In [ ]:
!pip install roboflow

In [ ]:
# Modelo YOLOv5 para Detecção de Cães e Gatos
# Dataset: https://universe.roboflow.com/dogandcat-tojsg/dog-and-cat-eu2ol/dataset/1

import os
import yaml
import torch
import roboflow
from roboflow import Roboflow
from google.colab import userdata
import subprocess
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import numpy as np
import time
import re

In [ ]:
# ===== CONFIGURAÇÃO E DOWNLOAD DO DATASET =====
def setup_dataset():
    """Configurar e baixar o dataset do Roboflow"""
    print("🔑 Configurando acesso ao Roboflow...")

    # Sua configuração inicial
    minha_chave_api = userdata.get('Roboflow')
    rf = Roboflow(api_key=minha_chave_api)

    # Acessar o projeto
    project = rf.workspace("dogandcat-tojsg").project("dog-and-cat-eu2ol")
    version = project.version(1)

    # Download do dataset no formato YOLOv5
    print("📥 Baixando dataset...")
    dataset = version.download("yolov5")

    return dataset.location

In [ ]:
# ===== INSTALAÇÃO E CONFIGURAÇÃO DO YOLOv5 =====
def setup_yolo():
    """Instalar e configurar YOLOv5"""
    print("🚀 Configurando YOLOv5...")

    # Clone YOLOv5 repository
    if not os.path.exists('yolov5'):
        subprocess.run(['git', 'clone', 'https://github.com/ultralytics/yolov5'], check=True)

    # Install requirements
    subprocess.run(['pip', 'install', '-r', 'yolov5/requirements.txt'], check=True)

    os.chdir('yolov5')
    return True

In [ ]:
# ===== CONFIGURAÇÃO DO ARQUIVO YAML =====
def create_config_file(dataset_path):
    """Criar arquivo de configuração YAML para o dataset"""
    print("📝 Criando arquivo de configuração...")

    config = {
        'train': os.path.join(dataset_path, 'train/images'),
        'val': os.path.join(dataset_path, 'valid/images'),
        'test': os.path.join(dataset_path, 'test/images'),
        'nc': 2,  # número de classes
        'names': ['cat', 'dog']  # nomes das classes
    }

    config_path = 'dog_cat_config.yaml'
    with open(config_path, 'w') as f:
        yaml.dump(config, f, default_flow_style=False)

    print(f"✅ Arquivo de configuração criado: {config_path}")
    return config_path

In [ ]:
# ===== TREINAMENTO DO MODELO =====
def train_model(config_path, epochs=100, img_size=640, batch_size=16, verbose=True):
    """Treinar o modelo YOLOv5 com progresso visível"""
    print("🏋️ Iniciando treinamento...")
    print(f"📊 Configurações: {epochs} épocas, batch={batch_size}, img_size={img_size}")
    print("="*60)

    # Comando de treinamento
    train_cmd = [
        'python', 'train.py',
        '--img', str(img_size),
        '--batch', str(batch_size),
        '--epochs', str(epochs),
        '--data', config_path,
        '--weights', 'yolov5s.pt',  # usar pesos pré-treinados
        '--name', 'dog_cat_model',
        '--cache',
        '--project', 'runs/train'  # especificar diretório
    ]

    if verbose:
        # Executar com output em tempo real
        print("🚀 Treinamento iniciado - acompanhe o progresso:")
        print("="*60)

        process = subprocess.Popen(train_cmd, stdout=subprocess.PIPE,
                                 stderr=subprocess.STDOUT, text=True, bufsize=1)

        # Monitorar progresso linha por linha
        epoch_pattern = r'Epoch\s+(\d+)/(\d+)'
        batch_pattern = r'(\d+)/(\d+)\s+\[.*?\]'

        for line in process.stdout:
            line = line.strip()
            if line:
                # Destacar informações importantes
                if 'Epoch' in line and ('train' in line or 'val' in line):
                    print(f"📈 {line}")
                elif 'mAP@0.5' in line or 'precision' in line or 'recall' in line:
                    print(f"📊 {line}")
                elif 'Best' in line:
                    print(f"🎯 {line}")
                elif 'Optimizer' in line:
                    print(f"⚙️ {line}")
                elif 'wandb' in line or 'tensorboard' in line:
                    print(f"📋 {line}")
                else:
                    # Mostrar outras linhas importantes
                    if any(keyword in line.lower() for keyword in
                          ['epoch', 'loss', 'map', 'precision', 'recall', 'speed']):
                        print(f"   {line}")

        process.wait()
        return_code = process.returncode

    else:
        # Executar sem output detalhado
        result = subprocess.run(train_cmd, capture_output=True, text=True)
        return_code = result.returncode

    if return_code == 0:
        print("\n" + "="*60)
        print("✅ TREINAMENTO CONCLUÍDO COM SUCESSO! 🎉")
        print("="*60)
        return 'runs/train/dog_cat_model'
    else:
        print("\n" + "="*60)
        print("❌ ERRO NO TREINAMENTO:")
        print("="*60)
        if not verbose:
            print(result.stderr)
        return None

In [ ]:
# ===== VALIDAÇÃO DO MODELO =====
def validate_model(model_path, config_path):
    """Validar o modelo treinado"""
    print("🔍 Validando modelo...")

    val_cmd = [
        'python', 'val.py',
        '--weights', f'{model_path}/weights/best.pt',
        '--data', config_path,
        '--img', '640',
        '--name', 'dog_cat_validation'
    ]

    subprocess.run(val_cmd)
    print("✅ Validação concluída!")

In [ ]:
# ===== FUNÇÃO DE PREDIÇÃO =====
def predict_image(model_path, image_path, conf_threshold=0.25):
    """Fazer predições em uma imagem"""

    # Carregar modelo
    model = torch.hub.load('ultralytics/yolov5', 'custom',
                          path=f'{model_path}/weights/best.pt', force_reload=True)

    # Configurar threshold de confiança
    model.conf = conf_threshold

    # Fazer predição
    results = model(image_path)

    # Mostrar resultados
    results.show()
    results.save()

    return results

In [ ]:
# ===== MONITORAMENTO DE TREINAMENTO EM TEMPO REAL =====
def monitor_training_progress(log_file_path):
    """Monitorar progresso do treinamento lendo arquivo de log"""

    if not os.path.exists(log_file_path):
        print("❌ Arquivo de log não encontrado")
        return

    print("📊 Monitorando treinamento em tempo real...")
    print("="*60)

    with open(log_file_path, 'r') as f:
        f.seek(0, 2)  # Ir para o final do arquivo

        while True:
            line = f.readline()
            if line:
                if 'Epoch' in line:
                    print(f"📈 {line.strip()}")
                elif any(metric in line for metric in ['mAP', 'precision', 'recall', 'loss']):
                    print(f"📊 {line.strip()}")
            else:
                time.sleep(1)  # Aguardar novas linhas

In [ ]:
# ===== CLASSE PARA MONITORAMENTO AVANÇADO =====
class TrainingMonitor:
    """Classe para monitorar treinamento com mais detalhes"""

    def __init__(self):
        self.metrics = {
            'epochs': [],
            'train_loss': [],
            'val_loss': [],
            'map50': [],
            'map95': [],
            'precision': [],
            'recall': []
        }

    def parse_log_line(self, line):
        """Parse de linha de log para extrair métricas"""
        # Regex patterns para diferentes métricas
        epoch_pattern = r'Epoch\s+(\d+)/(\d+)'
        metrics_pattern = r'(\d+)\s+(\d+)\s+([\d\.]+)\s+([\d\.]+)\s+([\d\.]+)\s+([\d\.]+)\s+([\d\.]+)\s+([\d\.]+)'

        if 'Epoch' in line:
            match = re.search(epoch_pattern, line)
            if match:
                current_epoch = int(match.group(1))
                total_epochs = int(match.group(2))
                return {'type': 'epoch', 'current': current_epoch, 'total': total_epochs}

        return None

    def display_progress_bar(self, current, total, prefix='', suffix='', length=50):
        """Exibir barra de progresso"""
        percent = ("{0:.1f}").format(100 * (current / float(total)))
        filled_length = int(length * current // total)
        bar = '█' * filled_length + '-' * (length - filled_length)
        print(f'\r{prefix} |{bar}| {percent}% {suffix}', end='\r')

        if current == total:
            print()  # Nova linha quando completo

In [ ]:
# ===== TREINAMENTO COM CALLBACKS VISUAIS =====
def train_with_visual_feedback(config_path, epochs=100, img_size=640, batch_size=16):
    """Treinamento com feedback visual aprimorado"""

    print("🚀 INICIANDO TREINAMENTO COM MONITORAMENTO VISUAL")
    print("="*70)
    print(f"📊 Configurações:")
    print(f"   • Épocas: {epochs}")
    print(f"   • Batch Size: {batch_size}")
    print(f"   • Tamanho da Imagem: {img_size}")
    print(f"   • Dataset: {config_path}")
    print("="*70)

    # Comando de treinamento
    train_cmd = [
        'python', 'train.py',
        '--img', str(img_size),
        '--batch', str(batch_size),
        '--epochs', str(epochs),
        '--data', config_path,
        '--weights', 'yolov5s.pt',
        '--name', 'dog_cat_model',
        '--cache',
        '--project', 'runs/train',
        '--exist-ok'  # Permitir sobrescrever
    ]

    # Inicializar monitor
    monitor = TrainingMonitor()

    print("🔄 Treinamento em andamento...")
    print("="*70)

    try:
        # Executar processo com output em tempo real
        process = subprocess.Popen(
            train_cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True,
            bufsize=1,
            universal_newlines=True
        )

        current_epoch = 0
        total_epochs = epochs

        for line in iter(process.stdout.readline, ''):
            line = line.strip()

            if not line:
                continue

            # Parse da linha
            parsed = monitor.parse_log_line(line)

            if parsed and parsed['type'] == 'epoch':
                current_epoch = parsed['current']
                total_epochs = parsed['total']

                # Mostrar progresso geral
                progress_percent = (current_epoch / total_epochs) * 100
                print(f"\n🏃‍♂️ ÉPOCA {current_epoch}/{total_epochs} ({progress_percent:.1f}%)")
                monitor.display_progress_bar(current_epoch, total_epochs,
                                           prefix='Progresso Geral:',
                                           suffix=f'Época {current_epoch}')

            # Destacar linhas importantes
            if any(keyword in line.lower() for keyword in
                  ['epoch', 'train:', 'val:']):
                if 'train:' in line:
                    print(f"🔵 TREINO: {line}")
                elif 'val:' in line:
                    print(f"🟢 VALIDAÇÃO: {line}")
                elif 'Epoch' in line and 'GPU_mem' in line:
                    print(f"⚡ STATUS: {line}")

            # Mostrar métricas importantes
            if any(metric in line for metric in ['mAP@0.5:', 'mAP@0.5:0.95:']):
                print(f"🎯 MÉTRICAS: {line}")

            # Mostrar quando salva modelo
            if 'Best' in line and ('mAP' in line or 'fitness' in line):
                print(f"🏆 MELHOR MODELO: {line}")

            # Mostrar velocidade e tempo estimado
            if 'ETA' in line or 'left' in line:
                print(f"⏱️  TEMPO: {line}")

        process.wait()
        return_code = process.returncode

        print("\n" + "="*70)

        if return_code == 0:
            print("🎉 TREINAMENTO CONCLUÍDO COM SUCESSO!")
            print("🏆 Modelo salvo em: runs/train/dog_cat_model/weights/")
            return 'runs/train/dog_cat_model'
        else:
            print("❌ ERRO NO TREINAMENTO")
            return None

    except KeyboardInterrupt:
        print("\n⏹️  Treinamento interrompido pelo usuário")
        process.terminate()
        return None
    except Exception as e:
        print(f"\n❌ Erro inesperado: {str(e)}")
        return None

    finally:
        print("="*70)

In [ ]:
# ===== ANÁLISE DE RESULTADOS =====
def analyze_results(model_path):
    """Analisar e plotar resultados do treinamento"""
    print("📊 Analisando resultados...")

    results_path = f'{model_path}/results.png'
    if os.path.exists(results_path):
        img = Image.open(results_path)
        plt.figure(figsize=(15, 10))
        plt.imshow(img)
        plt.axis('off')
        plt.title('Resultados do Treinamento')
        plt.show()

    # Mostrar matriz de confusão se disponível
    confusion_matrix_path = f'{model_path}/confusion_matrix.png'
    if os.path.exists(confusion_matrix_path):
        img = Image.open(confusion_matrix_path)
        plt.figure(figsize=(15, 10))
        plt.imshow(img)
        plt.axis('off')
        plt.title('Matriz de Confusão')
        plt.show()

In [ ]:
# ===== FUNÇÃO PRINCIPAL =====
def main():
    """Função principal para executar todo o pipeline"""
    try:
        # 1. Setup do dataset
        dataset_path = setup_dataset()
        print(f"📁 Dataset baixado em: {dataset_path}")

        # 2. Setup do YOLOv5
        setup_yolo()

        # 3. Criar arquivo de configuração
        config_path = create_config_file(dataset_path)

        # 4. Treinar modelo com monitoramento visual
        print("\n🎯 Iniciando treinamento com monitoramento...")
        model_path = train_with_visual_feedback(config_path, epochs=50, batch_size=16)

        if model_path:
            # 5. Validar modelo
            validate_model(model_path, config_path)

            # 6. Analisar resultados
            analyze_results(model_path)

            print(f"🎉 Modelo treinado com sucesso!")
            print(f"📁 Modelo salvo em: {model_path}/weights/best.pt")

            return model_path

    except Exception as e:
        print(f"❌ Erro: {str(e)}")
        return None

In [ ]:
# ===== EXEMPLO DE USO =====
if __name__ == "__main__":
    # Executar pipeline completo
    model_path = main()

    # Exemplo de como usar o modelo para predições
    if model_path:
        print("\n" + "="*50)
        print("EXEMPLO DE PREDIÇÃO:")
        print("="*50)

        # Exemplo de predição (substitua pelo caminho da sua imagem)
        # image_path = "caminho/para/sua/imagem.jpg"
        # results = predict_image(model_path, image_path)

        print("Para fazer uma predição, use:")
        print("results = predict_image(model_path, 'caminho/para/imagem.jpg')")